# DAISY RNAseq data projection

This notebook project RNA sequencing data into a Latent Space representing gene modules that are co expressed given various conditions, cell types, ... 

Sakaiza Rasolofomanana Rajery

12/19/2024

## load packages

In [16]:
library(tidyverse,warn.conflicts=FALSE)
library(reticulate)
library(MASS,warn.conflicts=FALSE)
library(devtools,warn.conflicts=FALSE)
library(PLIER,warn.conflicts=FALSE)
library(reticulate)
library("ggpubr")
#install_github("wgmao/PLIER")

In [17]:
# srouce plier_util to get function GetnewdataB 
source('scripts/plier_util.R')

In [18]:
# use GetOrderedRowNormEM function from Marc to order matrices
GetOrderedRowNormEM <- function(exprs.mat, plier.model) {

    require(PLIER)
    
    # Z-score normalization
    exprs.norm <- rowNorm(exprs.mat)
    exprs.norm <- na.omit(exprs.norm)
    
    z.mat <- plier.model$Z
    genes.in.model <- rownames(z.mat)
    genes_in_exprs <- rownames(exprs.norm)
    
    # Find the common genes
    common_genes <- intersect(genes_in_exprs, genes.in.model)
    
    # Filter the matrices based on the common genes
    exprs.norm.filtered <- exprs.norm[common_genes, , drop = FALSE]
    z.mat.filtered <- z.mat[common_genes, , drop = FALSE]
        
  # Update the plier.model with the new Z matrix
  plier.model$Z <- z.mat.filtered

  # Return the updated plier.model and the filtered exprs.norm
  list(plier.model = plier.model, exprs.norm.filtered = exprs.norm.filtered)

}

## load data: RNA seq, Matrices, name to ID, metada

### RNA_seq data: chose visit timepoint 1 or 2 or timepoint difference

In [19]:
#tp <- "1"
#tp <- "2" 
tp <- "diff"

### load data

In [22]:
# path to rna dataset
path_to_rna <- "/Users/rasolofs/Library/CloudStorage/OneDrive-TheUniversityofColoradoDenver/DAISY RNA Phenoplier"

In [20]:
# create difference dataset and save in RNA Daisy folder on drive (just run once)

RNAseq_1_res <- as.matrix(readRDS(paste0(path_to_rna,"/RNA_Visit_2_Residuals.RDS")))
RNAseq_2_res <- as.matrix(readRDS(paste0(path_to_rna,"/RNA_Visit_1_Residuals.RDS")))
RNAseq_diff <- RNAseq_2_res - RNAseq_1_res

saveRDS(RNAseq_diff,paste0(path_to_rna,"/RNA_Visit_diff_Residuals.RDS")) 

In [21]:
# path to rna dataset
path_to_rna <- "/Users/rasolofs/Library/CloudStorage/OneDrive-TheUniversityofColoradoDenver/DAISY RNA Phenoplier"
#r-log adjusted for age, sex and ancestry at visit 1 and 2
RNAseq_data <- as.matrix(readRDS(paste0(path_to_rna,"/RNA_Visit_",tp,"_Residuals.RDS")))



#RNA_annot <- readRDS("Expression_Annotation.RDS")
RNA_pheno <- read_csv(paste0(path_to_rna,"/DAISY_RNASeq_Phenotype_Deidentified.csv"))

str(RNAseq_data)
head(RNAseq_data,5)
dim(RNAseq_data)

Rows: 136 Columns: 22
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): ID, Group
dbl (20): Freezer_Time, Female_YN, Multiple_DAISY_Baseline, S_W, NHW_YN, dr3...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


 num [1:22236, 1:136] 0.157 -0.203 -0.117 -0.308 -0.367 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:22236] "ENSG00000000003" "ENSG00000000419" "ENSG00000000457" "ENSG00000000460" ...
  ..$ : chr [1:136] "00041-0" "00110-0" "00139-0" "00159-0" ...


,00041-0,00110-0,00139-0,00159-0,00174-0,00177-0,00181-0,00234-0,00250-0,00266-0,⋯,39376-0,39390-0,50136-0,62291-0,81801-0,84002-0,84689-0,84731-0,84860-0,85365-0
ENSG00000000003,0.1571997,0.08089299,-0.30387341,-0.1281056,-0.02556462,-0.001815018,-0.04624763,0.331580182,-0.04364535,0.04202482,⋯,-0.002210014,-0.05451790,-0.3291559,-0.275097960,-0.01336361,-0.163365055,0.2340031,0.10324136,-0.04909108,0.09996736
ENSG00000000419,-0.2032435,0.44913263,0.30760147,0.1695176,0.27566668,-0.969017010,-0.26320729,-0.034463613,-0.36278291,0.02797521,⋯,0.202623631,-0.21847065,-0.2846322,0.073150662,-0.27284434,0.205517696,0.0847536,-0.07399179,0.25284476,0.15461088
ENSG00000000457,-0.1168159,-0.07517545,0.11045258,0.1913833,-0.10947257,0.298236294,0.04725017,-0.011905739,0.20162429,0.03158399,⋯,0.071559553,-0.17778298,0.0402209,0.188624569,0.09107400,-0.141068344,-0.2230987,-0.03099280,0.15426959,0.36514810
ENSG00000000460,-0.3084525,-0.20569135,0.04308793,-0.2381819,0.04715758,-1.412240604,-0.12573203,0.001104009,-0.25762959,0.13224791,⋯,0.320264648,0.09144173,-0.0327244,0.005248543,0.19795205,0.170744425,0.5917766,-0.03660670,0.01961461,-0.10579198
ENSG00000000938,-0.3665634,0.26332126,-0.02515758,0.3050104,-0.15467022,-0.080098994,-0.16975991,-0.085735410,-0.17693529,-0.17165265,⋯,-0.081288740,0.02811155,0.2495640,0.461930629,-0.11667938,0.007394442,-0.1182536,-0.30525813,-0.38565354,0.20385482


[1] 22236   136

### name to ensemble_id file

In [22]:
#load pickle data to convert gene name to EnsemblID
py_data <- py_load_object("data/input/genes_mapping_id_to_name.pkl")
data_t <- data.frame(t(data.frame(py_data)))
colnames(data_t) <- "gene"
name_to_ID <- rownames_to_column(data_t, var = "EnsemblID")
head(name_to_ID,5)

,EnsemblID,gene
,<chr>,<chr>
1,ENSG00000000457,SCYL3
2,ENSG00000000938,FGR
3,ENSG00000000971,CFH
4,ENSG00000001036,FUCA2
5,ENSG00000001084,GCLC


### metadata file

In [23]:
# Import model metadata file to re-build plier model
metadata <- readRDS("data/input/multiplier_model_metadata.rds")
print(metadata)

$L1
[1] 120.5661

$L2
[1] 241.1322

$L3
[1] 0.01269676



### Multiplier matrices

In [24]:
# load matrices to rebuild plier model
# LV x samples
b_model <- readRDS("data/input/multiplier_model_b.rds")
head(b_model,5)
# gene x LV
z_model <- readRDS("data/input/multiplier_model_z.rds")
head(z_model,5)

,SRP000599.SRR013549,SRP000599.SRR013550,SRP000599.SRR013551,SRP000599.SRR013552,SRP000599.SRR013553,SRP000599.SRR013554,SRP000599.SRR013555,SRP000599.SRR013556,SRP000599.SRR013557,SRP000599.SRR013558,⋯,SRP035599.SRR1139372,SRP035599.SRR1139393,SRP035599.SRR1139388,SRP035599.SRR1139378,SRP035599.SRR1139399,SRP035599.SRR1139386,SRP035599.SRR1139375,SRP035599.SRR1139382,SRP035599.SRR1139356,SRP035599.SRR1139370
"1,REACTOME_MRNA_SPLICING",-0.059296689,-0.047909034,-0.049366085,-0.065078034,-0.036394186,-0.046432986,-0.040980500,-0.040068202,-0.046137392,-0.048547681,⋯,0.028219530,0.035137107,0.065077330,0.078143650,0.092361864,0.069042346,0.090913845,0.096341467,0.13111465,0.171751422
"2,SVM Monocytes",0.006212678,0.003625471,0.006604582,0.009258006,0.005061427,0.004132735,0.008950264,0.007226716,0.007240987,0.005709697,⋯,-0.050455152,-0.034501970,-0.033640290,-0.049702173,-0.037425739,-0.050069528,-0.022575052,-0.055091302,-0.05686929,-0.018072570
"3,REACTOME_TRANSMISSION_ACROSS_CHEMICAL_SYNAPSES",-0.026105335,-0.032232060,-0.020621382,-0.027598555,-0.035248076,-0.038700769,-0.032527087,-0.030592727,-0.028937277,-0.027405660,⋯,-0.028609689,-0.033449754,-0.030583001,-0.032399106,-0.029365381,-0.025405876,-0.033657228,-0.031317680,-0.03092424,-0.027868614
"4,REACTOME_NEURONAL_SYSTEM",-0.022079745,-0.008970910,-0.020341711,-0.016260213,-0.003022898,0.002442659,-0.020457842,-0.023735309,-0.021581483,-0.022477572,⋯,-0.037122216,-0.029658154,-0.036349546,-0.039253549,-0.035204624,-0.036345061,-0.034513880,-0.035925708,-0.04035837,-0.031131153
LV 5,0.007663157,0.007036176,0.006608393,0.003446311,0.006340665,0.007106127,0.007930485,0.009164026,0.008023601,0.007937586,⋯,-0.003055909,-0.004783739,-0.004352417,-0.004159541,-0.001084991,-0.001884109,-0.003561052,-0.003546184,-0.01210732,-0.001192709


GAS6,0.0000000,0,0.03943774,0,0.05047625,0.00000000,0,0.000000000,0.5909494,0.00000000,⋯,0.05012510,0.00000000,0.033407371,0.00000000,0.00000000,0.005963392,0.34736209,0,0.00000000,0.0000000
MMP14,0.0000000,0,0.00000000,0,0.07007159,0.00000000,0,0.004904131,1.7201788,2.42359463,⋯,0.00000000,0.00000000,0.001007286,0.00000000,0.03574724,0.000000000,0.00000000,0,0.01497801,0.0000000
DSP,0.0000000,0,0.00000000,0,0.00000000,0.04169683,0,0.005718149,0.0000000,0.00000000,⋯,0.02085321,0.00000000,0.000000000,0.00000000,0.00000000,0.005774440,0.00000000,0,0.00000000,0.4164045
MARCKSL1,0.3052117,0,0.00000000,0,0.00000000,0.00000000,0,0.000000000,0.1618435,0.14947148,⋯,0.02713418,0.05271997,0.000000000,0.03018947,0.06088351,0.000000000,0.00000000,0,0.00000000,0.4484800
SPARC,0.0000000,0,0.00000000,0,0.00000000,0.00000000,0,0.000000000,0.0000000,0.01401441,⋯,0.00000000,0.00000000,0.000000000,0.00000000,0.00000000,0.000000000,0.06777859,0,0.12241734,0.0626649


In [25]:
# see how many people among 136 were maintainer, reverter, progresser. 
patient_per_group <- RNA_pheno%>%
group_by(Group)%>%
summarize(n= n())

patient_per_group

Group,n
<chr>,<int>
Maintainer,64
Progressor,25
Reverter,47


## label z_matrix with ensemblID instead of gene name

In [26]:
# convert matrix to dataframe with gene as rownames
z_df <- cbind(gene = rownames(z_model), as.data.frame(z_model))
# merge dataframe with 'name_to_ID' (file containing name and ensembleID) by gene
EnsID_df <- merge(z_df, name_to_ID, by = "gene")
#set rownames of new dataframe to ensembleID
rownames(EnsID_df) <- EnsID_df$EnsemblID
#remove gene and ensembleID column
z_matrix <- as.matrix(EnsID_df[, !(colnames(EnsID_df) %in% c("gene", "EnsemblID"))])

str(z_matrix)
head(z_matrix,5)
dim(z_matrix)

 num [1:6455, 1:987] 0 0.2712 0.0051 0.513 1.0082 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:6455] "ENSG00000175899" "ENSG00000094914" "ENSG00000129673" "ENSG00000090861" ...
  ..$ : chr [1:987] "V1" "V2" "V3" "V4" ...


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V978,V979,V980,V981,V982,V983,V984,V985,V986,V987
ENSG00000175899,0.000000000,0.0000000,0.00000000,0.07842750,0,0.0000000,0.0000000,0.04677228,0.0000000,0.0110329,⋯,0.010910,0.00000000,0.052869436,0.012748534,0,0.01398231,0.15224090,0.00000000,0.0000000,0.10888358
ENSG00000094914,0.271162234,0.0000000,0.00000000,0.00000000,0,0.0000000,0.0000000,0.00000000,1.5133366,0.0000000,⋯,0.000000,0.00000000,0.000000000,0.001385031,0,0.00000000,0.71189565,0.03053447,0.4471055,0.42468522
ENSG00000129673,0.005098885,0.3084969,0.02881539,0.00000000,0,0.0000000,0.3717253,0.00000000,0.1352375,0.0000000,⋯,0.117163,0.01881498,0.000000000,0.193141835,0,0.04040077,0.18043565,0.00000000,0.0000000,0.04206449
ENSG00000090861,0.512978085,0.0000000,0.45128547,0.31956792,0,0.0000000,0.0000000,0.00000000,0.0000000,0.0000000,⋯,0.000000,0.00000000,0.006207784,0.021274385,0,0.00000000,0.06214269,0.00000000,0.0000000,0.36489213
ENSG00000124608,1.008156895,0.0000000,0.19742406,0.02483182,0,0.5708101,0.0000000,0.22867460,0.0000000,0.1162435,⋯,0.000000,0.00000000,0.153344992,0.000000000,0,0.00000000,1.00457811,0.01447016,0.0000000,0.00000000


[1] 6455  987

In [27]:
# rebuild plier model with z, b, L1, L2, L3
plier.model <- list(Z = z_matrix, B =b_model, L1 = metadata$L1, L2 = metadata$L2, L3 = metadata$L3)

In [28]:
# Check all variables are loaded
ls()

[1] "b_model"                      "CalculateHoldoutAUC"         
 [3] "CalculateUSparsity"           "data_t"                      
 [5] "EnsID_df"                     "EvalWrapper"                 
 [7] "EvalWrapperWithHoldout"       "GetNewDataB"                 
 [9] "GetOrderedRowNorm"            "GetOrderedRowNormEM"         
[11] "GetPathwayCoverage"           "GetReconstructedExprs"       
[13] "GetReconstructionCorrelation" "GetReconstructionMASE"       
[15] "LV_names"                     "metadata"                    
[17] "name_to_ID"                   "path_to_rna"                 
[19] "patient_per_group"            "plier.model"                 
[21] "PLIERNewData"                 "py_data"                     
[23] "remove_dot"                   "remove_pathway"              
[25] "RNA_pheno"                    "RNA_proj"                    
[27] "RNA_proj_group"               "RNA_proj_t"                  
[29] "RNAseq_1_res"                 "RNAseq_2_res"                
[31] "RNAseq_data"                  "RNAseq_diff"                 
[33] "tp"                           "z_df"                        
[35] "z_matrix"                     "z_model"

In [29]:
#order gene expression matrix to match plier model
ordered_mat <- GetOrderedRowNormEM(RNAseq_data, plier.model)

#extract gene_expressiom matrix
ordered_DAISY_gen_expr <- ordered_mat$exprs.norm.filtered

# extract plier model matrix
ordered_model <- ordered_mat$plier.model

In [30]:
#project gen expression matrix into plier model
projection <- GetNewDataB(ordered_DAISY_gen_expr,ordered_model)

In [31]:
#save projection
df_projection  <- as.data.frame(projection)
saveRDS(df_projection,paste0("output/projection_",tp,".rds"))

In [32]:
dim(df_projection)

[1] 987 136

In [33]:
head(df_projection, 10)

,00041-0,00110-0,00139-0,00159-0,00174-0,00177-0,00181-0,00234-0,00250-0,00266-0,⋯,39376-0,39390-0,50136-0,62291-0,81801-0,84002-0,84689-0,84731-0,84860-0,85365-0
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
"1,REACTOME_MRNA_SPLICING",0.065332401,-0.0426748090,-0.041625433,-0.04801535,-0.0442214261,0.004729490,0.071093981,0.006548052,-0.015402144,0.181385434,⋯,-0.008721237,0.131912850,0.0296017388,0.0739803592,-0.036742593,-0.046748298,-0.03986162,0.003378565,0.0776117098,-0.038489040
"2,SVM Monocytes",-0.011055519,-0.0203915426,-0.036730092,0.23206520,0.0056482973,0.090230557,-0.053618383,-0.055892209,0.033348187,-0.202543906,⋯,-0.054903032,0.161937335,0.0128464744,0.0483956407,0.014868334,0.058577340,0.02346133,-0.021035869,-0.2087083294,-0.158593876
"3,REACTOME_TRANSMISSION_ACROSS_CHEMICAL_SYNAPSES",-0.028714363,-0.0001745483,-0.046952233,-0.02635038,0.0019395758,0.034059247,-0.031111455,0.058060764,-0.004556753,-0.024825236,⋯,-0.013600209,0.032727480,-0.0230385126,-0.0005035420,-0.016334163,0.018761303,-0.10980930,-0.057315142,0.0050074827,-0.005712764
"4,REACTOME_NEURONAL_SYSTEM",0.036526145,0.0036821938,0.089392721,0.05435819,0.0120303486,-0.036533686,-0.002383970,0.006600057,0.037116839,-0.013348465,⋯,-0.018481614,-0.021814415,-0.0137323571,-0.1149671466,0.038998012,0.004115676,0.15045417,0.103095530,-0.0324800579,-0.039781879
LV 5,0.001051845,-0.0022437564,-0.049189611,0.01083836,-0.0095265953,-0.070493378,-0.018886612,0.033639892,0.034396240,0.022692889,⋯,0.003492555,0.008323240,0.0008771747,-0.0004055108,-0.007240147,0.011602828,0.05174336,-0.004552460,0.0268384648,0.028493582
LV 6,-0.004441117,0.0081585522,0.032472113,-0.02382115,-0.0119864145,0.026034243,0.005130581,0.038804318,0.053005779,0.029817126,⋯,-0.018442623,-0.011881862,0.0085084666,-0.0495462730,0.030367628,0.017500377,0.04229019,0.026824612,0.0002794303,-0.009901047
"7,IRIS_Neutrophil-Resting",-0.125406419,0.1186735279,-0.021726760,0.05710884,-0.0485965759,-0.003346754,-0.074511353,-0.030957254,-0.074895980,0.002175420,⋯,-0.033625884,-0.002532941,-0.0040099622,0.1833619070,-0.040083550,-0.109534077,-0.01662715,-0.115557188,-0.1152396948,0.172599313
"8,KEGG_PPAR_SIGNALING_PATHWAY",-0.016581769,0.0281342100,0.012381584,0.01861838,-0.0120970582,-0.092804899,0.035817899,0.027643473,0.012907509,0.004294152,⋯,-0.021820877,0.001497020,0.0296942684,-0.0210360620,-0.100706546,-0.023419421,0.01495798,-0.006865413,0.0115382280,-0.015575188
LV 9,-0.017315869,0.0131125908,-0.040387114,-0.03851220,0.0006907144,0.032555099,0.022087865,0.031584133,-0.049575864,-0.053893271,⋯,-0.016691451,0.088411748,0.0002051650,0.0488821876,-0.066955883,0.038332528,-0.17449992,-0.089385115,0.0088830595,0.047719623


In [34]:
# importing results from projection og DAISY RNAseq visit 1 into Phenoplier latent space
#RNA_proj <- readRDS("output/projection_1.rds")
RNA_proj <- readRDS(paste0("output/projection_",tp,".rds"))
dim(RNA_proj)
head(RNA_proj,1)

print(tp)


# transposing projection output data to get LV names, check we have 987 LV
RNA_proj_t <- data.frame(t(RNA_proj))
LV_names <- colnames(RNA_proj_t)
length(LV_names)
head(RNA_proj_t,1)

# convert matrix to dataframe with gene as rownames
RNA_proj_t <- cbind(ID = colnames(RNA_proj), t(RNA_proj))
# merge transposed RNA projection output with each individual's T1D groupping
RNA_proj_group <- merge(RNA_proj_t, RNA_pheno[,1:2], by = "ID")

head(RNA_proj_group,1)
#set rownames of new dataframe to ensembleID
#rownames(EnsID_df) <- EnsID_df$EnsemblID
#remove gene and ensembleID column
#z_matrix <- as.matrix(EnsID_df[, !(colnames(EnsID_df) %in% c("gene", "EnsemblID"))])

colnames(RNA_proj_group)[2:988] <- LV_names
#this <- RNA_proj_group[,LV_col]
#convert all LV col into numeric
RNA_proj_group[,LV_names] <- sapply(RNA_proj_group[,LV_names], function(x) as.numeric(as.character(x)))
head(RNA_proj_group,1) 

                                    # order groupping factor
RNA_proj_group$Group <- ordered(RNA_proj_group$Group,
                         levels = c("Maintainer", "Reverter", "Progressor"))

# correct column name 
#LV_names<- colnames(RNA_proj_group)[2:988]

remove_dot <- which(str_detect(LV_names, "LV."))
remove_pathway <- which(!str_detect(LV_names, "LV."))
LV_names[remove_pathway] <- gsub("\\..*","",LV_names[remove_pathway])
LV_names[remove_pathway] <- gsub("X","LV",LV_names[remove_pathway])
LV_names[remove_dot] <- gsub("\\.","",LV_names[remove_dot])

colnames(RNA_proj_group)[2:988]<- LV_names
head(RNA_proj_group,5)

saveRDS(RNA_proj_group,paste0("output/gp_RNA_proj_",tp,".rds"))

[1] 987 136

,00041-0,00110-0,00139-0,00159-0,00174-0,00177-0,00181-0,00234-0,00250-0,00266-0,⋯,39376-0,39390-0,50136-0,62291-0,81801-0,84002-0,84689-0,84731-0,84860-0,85365-0
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
"1,REACTOME_MRNA_SPLICING",0.0653324,-0.04267481,-0.04162543,-0.04801535,-0.04422143,0.00472949,0.07109398,0.006548052,-0.01540214,0.1813854,⋯,-0.008721237,0.1319128,0.02960174,0.07398036,-0.03674259,-0.0467483,-0.03986162,0.003378565,0.07761171,-0.03848904


[1] "diff"


[1] 987

,X1.REACTOME_MRNA_SPLICING,X2.SVM.Monocytes,X3.REACTOME_TRANSMISSION_ACROSS_CHEMICAL_SYNAPSES,X4.REACTOME_NEURONAL_SYSTEM,LV.5,LV.6,X7.IRIS_Neutrophil.Resting,X8.KEGG_PPAR_SIGNALING_PATHWAY,LV.9,X10.SVM.Macrophages.M0,⋯,LV.978,LV.979,LV.980,LV.981,LV.982,LV.983,LV.984,X985.IRIS_Neutrophil.Resting,LV.986,LV.987
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
00041-0,0.0653324,-0.01105552,-0.02871436,0.03652614,0.001051845,-0.004441117,-0.1254064,-0.01658177,-0.01731587,-0.02100245,⋯,0.0423164,0.03674118,0.02354014,0.02988662,-0.002217311,0.03846297,-0.06393674,-0.1348164,0.002138531,0.009733345


,ID,"1,REACTOME_MRNA_SPLICING","2,SVM Monocytes","3,REACTOME_TRANSMISSION_ACROSS_CHEMICAL_SYNAPSES","4,REACTOME_NEURONAL_SYSTEM",LV 5,LV 6,"7,IRIS_Neutrophil-Resting","8,KEGG_PPAR_SIGNALING_PATHWAY",LV 9,⋯,LV 979,LV 980,LV 981,LV 982,LV 983,LV 984,"985,IRIS_Neutrophil-Resting",LV 986,LV 987,Group
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,00041-0,0.0653324007929379,-0.0110555193273455,-0.0287143634855157,0.0365261447534386,0.00105184546723774,-0.00444111658164519,-0.125406419043393,-0.0165817685431146,-0.0173158694761706,⋯,0.0367411791341952,0.0235401357527444,0.0298866218343636,-0.0022173114513411,0.0384629727914095,-0.0639367439948502,-0.134816382592211,0.00213853131552632,0.00973334508517906,Maintainer


,ID,X1.REACTOME_MRNA_SPLICING,X2.SVM.Monocytes,X3.REACTOME_TRANSMISSION_ACROSS_CHEMICAL_SYNAPSES,X4.REACTOME_NEURONAL_SYSTEM,LV.5,LV.6,X7.IRIS_Neutrophil.Resting,X8.KEGG_PPAR_SIGNALING_PATHWAY,LV.9,⋯,LV.979,LV.980,LV.981,LV.982,LV.983,LV.984,X985.IRIS_Neutrophil.Resting,LV.986,LV.987,Group
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,00041-0,0.0653324,-0.01105552,-0.02871436,0.03652614,0.001051845,-0.004441117,-0.1254064,-0.01658177,-0.01731587,⋯,0.03674118,0.02354014,0.02988662,-0.002217311,0.03846297,-0.06393674,-0.1348164,0.002138531,0.009733345,Maintainer


,ID,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,⋯,LV979,LV980,LV981,LV982,LV983,LV984,LV985,LV986,LV987,Group
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<ord>
1,00041-0,0.06533240,-0.011055519,-0.0287143635,0.036526145,0.001051845,-0.004441117,-0.12540642,-0.01658177,-0.0173158695,⋯,0.036741179,0.023540136,0.029886622,-0.002217311,0.03846297,-0.06393674,-0.134816383,0.002138531,0.009733345,Maintainer
2,00110-0,-0.04267481,-0.020391543,-0.0001745483,0.003682194,-0.002243756,0.008158552,0.11867353,0.02813421,0.0131125908,⋯,-0.003676397,-0.034794733,-0.010491055,-0.007721971,-0.01465504,-0.14834049,0.008681795,0.015668703,0.035779363,Reverter
3,00139-0,-0.04162543,-0.036730092,-0.0469522326,0.089392721,-0.049189611,0.032472113,-0.02172676,0.01238158,-0.0403871139,⋯,-0.023873847,0.002156358,-0.010940347,0.010072032,-0.03348186,-0.08142656,0.123767615,0.001222313,0.039788403,Reverter
4,00159-0,-0.04801535,0.232065205,-0.0263503818,0.054358188,0.010838359,-0.023821148,0.05710884,0.01861838,-0.0385121968,⋯,0.023145471,-0.004236950,0.022258250,0.029260069,-0.02153985,-0.29057564,0.194484202,-0.055334761,-0.026108705,Maintainer
5,00174-0,-0.04422143,0.005648297,0.0019395758,0.012030349,-0.009526595,-0.011986414,-0.04859658,-0.01209706,0.0006907144,⋯,0.005872110,-0.009735373,0.005824244,-0.007314493,0.03117020,0.11755467,-0.011915534,0.009478003,-0.068822916,Progressor


In [ ]:
# boxplot one LV by group, and compute anova p-value
ggboxplot(RNA_proj_group, x = "Group", y = "LV986", color = "Group", 
          add = "jitter", legend = "none") +
  rotate_x_text(angle = 45)+
  geom_hline(yintercept = mean(RNA_proj_group$LV986), linetype = 2)+ # Add horizontal line at base mean
  stat_compare_means(method = "anova", label.y = 0.08)+        # Add global annova p-value
  stat_compare_means(method = "t.test",
                     ref.group = ".all.",label.y = c(0.10,0.10,0.10))


# compute anova for each LV

res <- data.frame(matrix(nrow = 0, ncol = 6))

# compare each group to the mean expression accross all three groups
for (i in 1:length(LV_names)){
    comp_i <- compare_means(as.formula(paste0(LV_names[i]," ~ Group")),  data = RNA_proj_group,
                            method = "anova")
    res <- rbind(res, comp_i)
    }
res_formated <- res[,c(1,2)]
colnames(res_formated) <- c("LV","p_val")
res_formated$fdr <- p.adjust(res_formated$p_val, method = "BH")
head(res_formated,5)

saveRDS(res_formated,paste0("output/anova_",tp,".rds"))